# Machine Learning Basics

**Abstract:** These exercises will familiarize you with the basics of doing regression and classification using scikit-learn.  You will also get an introduction to some techniques for estimating generalization performance.

Good background reading: ["A few useful things to know about machine learning"](http://homes.cs.washington.edu/%7Epedrod/papers/cacm12.pdf), 

Before beginning, you should verify that you have scikit-learn and matplotlib installed properly.  To do so, run the following code block and make sure that you don't get any import errors.  Additionally, the version of the sklearn module should be at least 0.13.

In [ ]:
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

print sklearn.__version__

## What is Machine Learning

Tom Mitchell defines what it means for a computer program to learn in the following way: “A computer program is said to learn from experience E with respect to some class of tasks T and performance measure P, if its performance at tasks in T, as measured by P, improves with experience E.”

This definition highlights a key difference between machine learning and classical statistical methods.  That is, machine learning is chiefly concerned with improving future performance based on prior experience.

Another key difference with classical statistical methods is that machine learning focuses on the computational efficiency (in both time and space) of algorithms.  For instance, an active area of machine learning research is to create algorithms that have computational efficiency properties that work with “big data”.

## Doing machine learning using scikit-learn

The specific software package we will be using to do machine learning is called [scikit-learn](http://scikit-learn.org/stable/).  Scikit-learn is a very powerful package that supports a vast array of machine learning algorithms.  To get a sense of the toolkits capabilities check out the [examples page](http://scikit-learn.org/stable/auto_examples/index.html).

## Training Versus Testing Performance

To better understand how machine learning differs from classical methods, we'll use an example taken directly from scikit learn's built-in datasets.

The first step will be to load a dataset to use for our analysis.  First, we will be working with a dataset of Boston real estate prices.  In order to load the data and print out a detailed description of this dataset use the following code:

In [ ]:
from sklearn.datasets import *

data = load_boston()
print data.DESCR

To learn a simple model of housing prices using multiple linear regression and print the quality of our model (for some yet to be defined notion of quality), we use this code.

In [ ]:
from sklearn.datasets import *
from sklearn.linear_model import LinearRegression

data = load_boston()
model = LinearRegression()
model.fit(data.data, data.target)

print model.score(data.data, data.target)

This code will tell us how well the model does on explaining the data we used to fit the model.  In machine learning, we focus on model performance on unseen data.  In order to estimate the performance of the system on unseen data, we can split the data into two sets: the training set and the test set.  The following code will fit a model just using the training data and print out the coefficient of determination for both the training and testing data.

In [ ]:
from sklearn.datasets import *
from sklearn.cross_validation import train_test_split

data = load_boston()

X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, train_size=0.5)
model = LinearRegression()
model.fit(X_train, y_train)

print "Train Quality %f"%model.score(X_train,y_train)
print "Test Quality %f"%model.score(X_test,y_test)

This simple experiment gets at the idea that estimating your model’s performance using the same data used for fitting is not an accurate predictor of how well the model will do on new data.

Further, there is a relationship between model complexity, amount of training data, and the gap between the performance of a model on the training data versus the testing data.

To get a better handle on this, we let's generate a plot showing the Quality of fit on the training and the test set versus the number of the 13 features used to fit the model.  For instance at the value of 3 on the x-axis a random subset of size 3 from the original 13 features was selected for learning.  This procedure was repeated 1,000 times to smooth out variability.

Questions:

1. Why do the curves look the way they do?

2. Does this remind you of any of the statistics we computed when we first saw multiple linear regression?

In [ ]:
import numpy as np

n_trials = 100

X = data.data
y = data.target

d = data.data.shape[1]

avg_train_quality = np.zeros(d)
avg_test_quality = np.zeros(d)

for n_features in range(1,d+1):
    quality_train = numpy.zeros(n_trials)
    quality_test = numpy.zeros(n_trials)

    for i in range(n_trials):
        # permute the columns of X
        permutation = np.random.permutation(range(d))
        X_subset = X[:,permutation[0:n_features]]

        X_train, X_test, y_train, y_test = train_test_split(X_subset, y, train_size=0.5)

        model = LinearRegression()
        model.fit(X_train, y_train)
        quality_train[i] = model.score(X_train, y_train)
        quality_test[i] = model.score(X_test,y_test)

    avg_train_quality[n_features-1] = quality_train.mean()
    avg_test_quality[n_features-1] = quality_test.mean()

fig = plt.figure()
train_line, = plt.plot(range(1,d+1),avg_train_quality)
plt.hold(True)
test_line, = plt.plot(range(1,d+1),avg_test_quality)
plt.legend([train_line, test_line],['Average Training Quality','Average Testing Quality'],loc="upper left")
plt.xlabel('Number of Features')
plt.show()

## Classification using scikit-learn

In addition to multiple regression, scikit-learn supports many other learning algorithms for both regression and classification.  In the classification setting, the goal is to assign a categorical label to an input rather than a continuous value (as in regression).  When doing classification, you will want to use both a evaluation function (to use the terminology in the Domingos paper) and a different learning algorithm.  To get started we will use an algorithm called multiple logistic regression that is built into scikit-learn.  Specifically, we will be building a model to classify images of handwritten digits.

To load the digits and display 10 of the examplars use the following code:

In [ ]:
from sklearn.datasets import *
import matplotlib.pyplot as plt
%matplotlib inline
import numpy

digits = load_digits()
print digits.DESCR
fig = plt.figure()
for i in range(10):
    subplot = fig.add_subplot(5,2,i+1)
    subplot.matshow(numpy.reshape(digits.data[i],(8,8)),cmap='gray')
plt.show()

Next, we will use [multinomial logistic regression](http://en.wikipedia.org/wiki/Multinomial_logistic_regression) to learn to classify images of digits based on their pixel brightnesses.  As before, we split the data into two sets in order to get an accurate estimate of how well our model will work on future images of digits.

In [ ]:
from sklearn.datasets import *
from sklearn.cross_validation import train_test_split

from sklearn.linear_model import LogisticRegression

data = load_digits()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, train_size=0.5)
model = LogisticRegression()
model.fit(X_train, y_train)
print "Train accuracy %f" %model.score(X_train,y_train)
print "Test accuracy %f"%model.score(X_test,y_test)

### Viewing Model Predictions

It's almost always a good idea to investigate the instances your model is getting right and those that it is getting wrong.

In [ ]:
def visualize_predictions(images, targets, predictions):
    """ Adapted from Jake Vanderplas' scikit learn tutorials. """
    fig, axes = plt.subplots(10, 10, figsize=(8, 8))
    fig.subplots_adjust(hspace=0.1, wspace=0.1)

    for i, ax in enumerate(axes.flat):
        ax.imshow(digits.images[i], cmap='binary')
        ax.text(0.05, 0.05, str(predictions[i]),
                transform=ax.transAxes,
                color='green' if (targets[i] == predictions[i]) else 'red')
        ax.set_xticks([])
        ax.set_yticks([])

visualize_predictions(digits.data,
                      digits.target,
                      model.predict(digits.data))

Use this code (or a modified version) to better understand the model's behavior.  Where is the model making accurate predictions, what about inaccurate ones?

## The relationship between the number of training examples and performance

Next, we will examine how the amount of training data influences the performance of the learned model.  Run the following code to generate a learning curve that shows the performance of the model on a testing set as a function of the amount of training examples used:

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn.datasets import *
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression

data = load_digits()
n_trials = 5
train_percentages = range(5,95,5)
test_accuracies = numpy.zeros(len(train_percentages))

for (i,train_percent) in enumerate(train_percentages):
        test_accuracy = numpy.zeros(n_trials)
        for n in range(n_trials):
                X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, train_size=train_percent/100.0)
                model = LogisticRegression()
                model.fit(X_train, y_train)
                test_accuracy[n] = model.score(X_test, y_test)
        test_accuracies[i] = test_accuracy.mean()

fig = plt.figure()
plt.plot(train_percentages, test_accuracies)
plt.xlabel('Percentage of Data Used for Training')
plt.ylabel('Accuracy on Test Set')
plt.show()

**Questions:**

1.  What is the general trend?

2.  Are there parts of the curve that appear to be noisier than others?  Why?

To reduce the noise in the curve, increase the number of repeated random trials by editing the code.